In [1]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width: 90% !important;}</style>"))

/var/folders/s2/bjtyvybn0kv3rwjkn5xht9_m0000gn/T/ipykernel_7753/2814205877.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display,HTML


In [2]:
import pandas as pd
import govt_func as gf
import datetime
from imp import reload

import numpy as np
from dateutil.relativedelta import *


# Import Data

In [3]:
deferred_uri = "cleaned_data/cleaned_deferred.csv"
df_deferred = pd.read_csv(deferred_uri)

nonfreq_uri = "cleaned_data/cleaned_nonfreq.csv"
df_nonfreq = pd.read_csv(nonfreq_uri)

other_uri = "cleaned_data/cleaned_other.csv"
df_other = pd.read_csv(other_uri)

return_uri = "cleaned_data/cleaned_return.csv"
df_return = pd.read_csv(return_uri)

capital_uri = "data/mkt_cap_month_freq.xlsx"
df_cap = pd.read_excel(capital_uri)

industry_ref = "data/industry_into_citics_1.txt"


# Get industry code mapping

In [4]:
file = open(industry_ref, "r")
industry_mapping = {}
line = file.readline().strip().split()
line = file.readline().strip().split()
while line != []:
    industry_mapping[line[1]] = int(line[0])
    line = file.readline().strip().split()
industry_mapping


{'石油石化': 10,
 '煤炭': 11,
 '有色金属': 12,
 '电力及公用事业': 20,
 '钢铁': 21,
 '基础化工': 22,
 '建筑': 23,
 '建材': 24,
 '轻工制造': 25,
 '机械': 26,
 '电力设备及新能源': 27,
 '国防军工': 28,
 '汽车': 30,
 '商贸零售': 31,
 '消费者服务': 32,
 '家电': 33,
 '纺织服装': 34,
 '医药': 35,
 '食品饮料': 36,
 '农林牧渔': 37,
 '银行': 40,
 '非银行金融': 41,
 '房地产': 42,
 '综合金融': 43,
 '交通运输': 50,
 '电子': 60,
 '通信': 61,
 '计算机': 62,
 '传媒': 63,
 '综合': 70}

# Merge All Factor Value

In [5]:
df_other = df_other.set_index(["QUARTER", "SECU_CODE", "INDUSTRY"])
df_nonfreq = df_nonfreq.set_index(["QUARTER", "SECU_CODE", "INDUSTRY"])
df = pd.merge(
    left=df_nonfreq,
    right=df_other,
    left_index=True,
    right_index=True,
    how="outer",
    suffixes=("_SUBSIDY", "_OTHER"),
)
df

F_SUPPORT_SUBSIDY  F_DEPENDENCE_SUBSIDY  \
QUARTER SECU_CODE INDUSTRY                                            
2011Q1  000009.SZ 综合                 1.458430             -0.543251   
        000012.SZ 建材                 1.629855             -0.437518   
        000016.SZ 家电                 1.497990              2.564561   
        000019.SZ 农林牧渔              -0.461529              2.564561   
        000020.SZ 电子                -1.517770             -0.547717   
...                                       ...                   ...   
2022Q3  688799.SH 医药                -0.114306             -0.570632   
        688800.SH 电子                -1.103734             -0.719448   
        688819.SH 电力设备及新能源           1.546257             -0.420753   
        688981.SH 电子                 1.696215             -0.365687   
        689009.SH 机械                -1.644024             -0.773081   

                            F_SUBSIDY  F_SUPPORT_OTHER  F_DEPENDENCE_OTHER  \
QUARTER SECU_CODE INDUSTRY                                                   
2011Q1  000009.SZ 综合         2.001681              NaN                 NaN   
        000012.SZ 建材         2.067373              NaN                 NaN   
        000016.SZ 家电        -1.066571              NaN                 NaN   
        000019.SZ 农林牧渔      -3.026089              NaN                 NaN   
        000020.SZ 电子        -0.970052              NaN                 NaN   
...                               ...              ...                 ...   
2022Q3  688799.SH 医药         0.456326        -0.146204           -0.615050   
        688800.SH 电子        -0.384286        -1.148131           -0.767875   
        688819.SH 电力设备及新能源   1.967011         1.506474           -0.471376   
        688981.SH 电子         2.061902         1.675612           -0.416715   
        689009.SH 机械        -0.870943        -0.861457           -0.796988   

                             F_OTHER  
QUARTER SECU_CODE INDUSTRY            
2011Q1  000009.SZ 综合             NaN  
        000012.SZ 建材             NaN  
        000016.SZ 家电             NaN  
        000019.SZ 农林牧渔           NaN  
        000020.SZ 电子             NaN  
...                              ...  
2022Q3  688799.SH 医药        0.468846  
        688800.SH 电子       -0.380256  
        688819.SH 电力设备及新能源  1.977850  
        688981.SH 电子        2.092327  
        689009.SH 机械       -0.064468  

[111638 rows x 6 columns]

In [6]:
df_deferred = df_deferred.set_index(["QUARTER", "SECU_CODE", "INDUSTRY"])
df = pd.merge(
    left=df, right=df_deferred, left_index=True, right_index=True, how="outer"
)
df

F_SUPPORT_SUBSIDY  F_DEPENDENCE_SUBSIDY  \
QUARTER SECU_CODE INDUSTRY                                            
2011Q1  000009.SZ 综合                 1.458430             -0.543251   
        000012.SZ 建材                 1.629855             -0.437518   
        000016.SZ 家电                 1.497990              2.564561   
        000019.SZ 农林牧渔              -0.461529              2.564561   
        000020.SZ 电子                -1.517770             -0.547717   
...                                       ...                   ...   
2022Q3  688799.SH 医药                -0.114306             -0.570632   
        688800.SH 电子                -1.103734             -0.719448   
        688819.SH 电力设备及新能源           1.546257             -0.420753   
        688981.SH 电子                 1.696215             -0.365687   
        689009.SH 机械                -1.644024             -0.773081   

                            F_SUBSIDY  F_SUPPORT_OTHER  F_DEPENDENCE_OTHER  \
QUARTER SECU_CODE INDUSTRY                                                   
2011Q1  000009.SZ 综合         2.001681              NaN                 NaN   
        000012.SZ 建材         2.067373              NaN                 NaN   
        000016.SZ 家电        -1.066571              NaN                 NaN   
        000019.SZ 农林牧渔      -3.026089              NaN                 NaN   
        000020.SZ 电子        -0.970052              NaN                 NaN   
...                               ...              ...                 ...   
2022Q3  688799.SH 医药         0.456326        -0.146204           -0.615050   
        688800.SH 电子        -0.384286        -1.148131           -0.767875   
        688819.SH 电力设备及新能源   1.967011         1.506474           -0.471376   
        688981.SH 电子         2.061902         1.675612           -0.416715   
        689009.SH 机械        -0.870943        -0.861457           -0.796988   

                             F_OTHER  F_SUPPORT  F_DEPENDENCE  F_DEFERRED  
QUARTER SECU_CODE INDUSTRY                                                 
2011Q1  000009.SZ 综合             NaN        NaN           NaN         NaN  
        000012.SZ 建材             NaN        NaN           NaN         NaN  
        000016.SZ 家电             NaN        NaN           NaN         NaN  
        000019.SZ 农林牧渔           NaN        NaN           NaN         NaN  
        000020.SZ 电子             NaN        NaN           NaN         NaN  
...                              ...        ...           ...         ...  
2022Q3  688799.SH 医药        0.468846   1.198285      0.333809    0.864475  
        688800.SH 电子       -0.380256   0.698050     -0.312493    1.010544  
        688819.SH 电力设备及新能源  1.977850   1.362780      0.031121    1.331659  
        688981.SH 电子        2.092327   0.767223      0.246840    0.520383  
        689009.SH 机械       -0.064468  -0.029103     -0.666340    0.637237  

[113747 rows x 9 columns]

In [7]:
df_gov = df.rename(
    columns={"F_SUPPORT": "F_SUPPORT_DEFERRED",
             "F_DEPENDENCE": "F_DEPENDENCE_DEFERRED"}
)
df_gov


F_SUPPORT_SUBSIDY  F_DEPENDENCE_SUBSIDY  \
QUARTER SECU_CODE INDUSTRY                                            
2011Q1  000009.SZ 综合                 1.458430             -0.543251   
        000012.SZ 建材                 1.629855             -0.437518   
        000016.SZ 家电                 1.497990              2.564561   
        000019.SZ 农林牧渔              -0.461529              2.564561   
        000020.SZ 电子                -1.517770             -0.547717   
...                                       ...                   ...   
2022Q3  688799.SH 医药                -0.114306             -0.570632   
        688800.SH 电子                -1.103734             -0.719448   
        688819.SH 电力设备及新能源           1.546257             -0.420753   
        688981.SH 电子                 1.696215             -0.365687   
        689009.SH 机械                -1.644024             -0.773081   

                            F_SUBSIDY  F_SUPPORT_OTHER  F_DEPENDENCE_OTHER  \
QUARTER SECU_CODE INDUSTRY                                                   
2011Q1  000009.SZ 综合         2.001681              NaN                 NaN   
        000012.SZ 建材         2.067373              NaN                 NaN   
        000016.SZ 家电        -1.066571              NaN                 NaN   
        000019.SZ 农林牧渔      -3.026089              NaN                 NaN   
        000020.SZ 电子        -0.970052              NaN                 NaN   
...                               ...              ...                 ...   
2022Q3  688799.SH 医药         0.456326        -0.146204           -0.615050   
        688800.SH 电子        -0.384286        -1.148131           -0.767875   
        688819.SH 电力设备及新能源   1.967011         1.506474           -0.471376   
        688981.SH 电子         2.061902         1.675612           -0.416715   
        689009.SH 机械        -0.870943        -0.861457           -0.796988   

                             F_OTHER  F_SUPPORT_DEFERRED  \
QUARTER SECU_CODE INDUSTRY                                 
2011Q1  000009.SZ 综合             NaN                 NaN   
        000012.SZ 建材             NaN                 NaN   
        000016.SZ 家电             NaN                 NaN   
        000019.SZ 农林牧渔           NaN                 NaN   
        000020.SZ 电子             NaN                 NaN   
...                              ...                 ...   
2022Q3  688799.SH 医药        0.468846            1.198285   
        688800.SH 电子       -0.380256            0.698050   
        688819.SH 电力设备及新能源  1.977850            1.362780   
        688981.SH 电子        2.092327            0.767223   
        689009.SH 机械       -0.064468           -0.029103   

                            F_DEPENDENCE_DEFERRED  F_DEFERRED  
QUARTER SECU_CODE INDUSTRY                                     
2011Q1  000009.SZ 综合                          NaN         NaN  
        000012.SZ 建材                          NaN         NaN  
        000016.SZ 家电                          NaN         NaN  
        000019.SZ 农林牧渔                        NaN         NaN  
        000020.SZ 电子                          NaN         NaN  
...                                           ...         ...  
2022Q3  688799.SH 医药                     0.333809    0.864475  
        688800.SH 电子                    -0.312493    1.010544  
        688819.SH 电力设备及新能源               0.031121    1.331659  
        688981.SH 电子                     0.246840    0.520383  
        689009.SH 机械                    -0.666340    0.637237  

[113747 rows x 9 columns]

# Synthesize Factor F_GOV

In [8]:
df = df_gov
mean = df.groupby("QUARTER").mean()
std = df.groupby("QUARTER").std()
df1 = df.reset_index().set_index("QUARTER")
df = pd.merge(
    left=df1,
    right=mean,
    how="outer",
    left_index=True,
    right_index=True,
    suffixes=["", "_mean"],
)
df = pd.merge(
    left=df,
    right=std,
    how="outer",
    left_index=True,
    right_index=True,
    suffixes=["", "_std"],
)
df.columns


Index(['SECU_CODE', 'INDUSTRY', 'F_SUPPORT_SUBSIDY', 'F_DEPENDENCE_SUBSIDY',
       'F_SUBSIDY', 'F_SUPPORT_OTHER', 'F_DEPENDENCE_OTHER', 'F_OTHER',
       'F_SUPPORT_DEFERRED', 'F_DEPENDENCE_DEFERRED', 'F_DEFERRED',
       'F_SUPPORT_SUBSIDY_mean', 'F_DEPENDENCE_SUBSIDY_mean', 'F_SUBSIDY_mean',
       'F_SUPPORT_OTHER_mean', 'F_DEPENDENCE_OTHER_mean', 'F_OTHER_mean',
       'F_SUPPORT_DEFERRED_mean', 'F_DEPENDENCE_DEFERRED_mean',
       'F_DEFERRED_mean', 'F_SUPPORT_SUBSIDY_std', 'F_DEPENDENCE_SUBSIDY_std',
       'F_SUBSIDY_std', 'F_SUPPORT_OTHER_std', 'F_DEPENDENCE_OTHER_std',
       'F_OTHER_std', 'F_SUPPORT_DEFERRED_std', 'F_DEPENDENCE_DEFERRED_std',
       'F_DEFERRED_std'],
      dtype='object')

In [9]:
df["F_SUBSIDY"] = (df["F_SUBSIDY"] - df["F_SUBSIDY_mean"]) / df["F_SUBSIDY_std"]
df["F_OTHER"] = (df["F_OTHER"] - df["F_OTHER_mean"]) / df["F_OTHER_std"]
df["F_DEFERRED"] = (df["F_DEFERRED"] - df["F_DEFERRED_mean"]) / df["F_DEFERRED_std"]
df = df.fillna(0)
df["F_GOV"] = df["F_SUBSIDY"] + df["F_OTHER"] + df["F_DEFERRED"]
df

,SECU_CODE,INDUSTRY,F_SUPPORT_SUBSIDY,F_DEPENDENCE_SUBSIDY,F_SUBSIDY,F_SUPPORT_OTHER,F_DEPENDENCE_OTHER,F_OTHER,F_SUPPORT_DEFERRED,F_DEPENDENCE_DEFERRED,...,F_SUPPORT_SUBSIDY_std,F_DEPENDENCE_SUBSIDY_std,F_SUBSIDY_std,F_SUPPORT_OTHER_std,F_DEPENDENCE_OTHER_std,F_OTHER_std,F_SUPPORT_DEFERRED_std,F_DEPENDENCE_DEFERRED_std,F_DEFERRED_std,F_GOV
QUARTER,,,,,,,,,,,,,,,,,,,,,
2011Q1,000009.SZ,综合,1.458430,-0.543251,1.551145,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.290454,0.0,0.0,0.000000,0.0,0.0,0.000000,1.551145
2011Q1,000012.SZ,建材,1.629855,-0.437518,1.602051,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.290454,0.0,0.0,0.000000,0.0,0.0,0.000000,1.602051
2011Q1,000016.SZ,家电,1.497990,2.564561,-0.826508,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.290454,0.0,0.0,0.000000,0.0,0.0,0.000000,-0.826508
2011Q1,000019.SZ,农林牧渔,-0.461529,2.564561,-2.344981,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.290454,0.0,0.0,0.000000,0.0,0.0,0.000000,-2.344981
2011Q1,000020.SZ,电子,-1.517770,-0.547717,-0.751714,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.290454,0.0,0.0,0.000000,0.0,0.0,0.000000,-0.751714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022Q3,688799.SH,医药,-0.114306,-0.570632,0.327839,-0.146204,-0.615050,0.337911,1.198285,0.333809,...,1.0,1.0,1.391923,1.0,1.0,1.387483,1.0,1.0,0.938795,1.586585
2022Q3,688800.SH,电子,-1.103734,-0.719448,-0.276083,-1.148131,-0.767875,-0.274062,0.698050,-0.312493,...,1.0,1.0,1.391923,1.0,1.0,1.387483,1.0,1.0,0.938795,0.526282
2022Q3,688819.SH,电力设备及新能源,1.546257,-0.420753,1.413161,1.506474,-0.471376,1.425494,1.362780,0.031121,...,1.0,1.0,1.391923,1.0,1.0,1.387483,1.0,1.0,0.938795,4.257132


In [10]:
df = df.reset_index().set_index(["QUARTER", "SECU_CODE", "INDUSTRY"])
df

F_SUPPORT_SUBSIDY  F_DEPENDENCE_SUBSIDY  \
QUARTER SECU_CODE INDUSTRY                                            
2011Q1  000009.SZ 综合                 1.458430             -0.543251   
        000012.SZ 建材                 1.629855             -0.437518   
        000016.SZ 家电                 1.497990              2.564561   
        000019.SZ 农林牧渔              -0.461529              2.564561   
        000020.SZ 电子                -1.517770             -0.547717   
...                                       ...                   ...   
2022Q3  688799.SH 医药                -0.114306             -0.570632   
        688800.SH 电子                -1.103734             -0.719448   
        688819.SH 电力设备及新能源           1.546257             -0.420753   
        688981.SH 电子                 1.696215             -0.365687   
        689009.SH 机械                -1.644024             -0.773081   

                            F_SUBSIDY  F_SUPPORT_OTHER  F_DEPENDENCE_OTHER  \
QUARTER SECU_CODE INDUSTRY                                                   
2011Q1  000009.SZ 综合         1.551145         0.000000            0.000000   
        000012.SZ 建材         1.602051         0.000000            0.000000   
        000016.SZ 家电        -0.826508         0.000000            0.000000   
        000019.SZ 农林牧渔      -2.344981         0.000000            0.000000   
        000020.SZ 电子        -0.751714         0.000000            0.000000   
...                               ...              ...                 ...   
2022Q3  688799.SH 医药         0.327839        -0.146204           -0.615050   
        688800.SH 电子        -0.276083        -1.148131           -0.767875   
        688819.SH 电力设备及新能源   1.413161         1.506474           -0.471376   
        688981.SH 电子         1.481334         1.675612           -0.416715   
        689009.SH 机械        -0.625712        -0.861457           -0.796988   

                             F_OTHER  F_SUPPORT_DEFERRED  \
QUARTER SECU_CODE INDUSTRY                                 
2011Q1  000009.SZ 综合        0.000000            0.000000   
        000012.SZ 建材        0.000000            0.000000   
        000016.SZ 家电        0.000000            0.000000   
        000019.SZ 农林牧渔      0.000000            0.000000   
        000020.SZ 电子        0.000000            0.000000   
...                              ...                 ...   
2022Q3  688799.SH 医药        0.337911            1.198285   
        688800.SH 电子       -0.274062            0.698050   
        688819.SH 电力设备及新能源  1.425494            1.362780   
        688981.SH 电子        1.508001            0.767223   
        689009.SH 机械       -0.046464           -0.029103   

                            F_DEPENDENCE_DEFERRED  F_DEFERRED  \
QUARTER SECU_CODE INDUSTRY                                      
2011Q1  000009.SZ 综合                     0.000000    0.000000   
        000012.SZ 建材                     0.000000    0.000000   
        000016.SZ 家电                     0.000000    0.000000   
        000019.SZ 农林牧渔                   0.000000    0.000000   
        000020.SZ 电子                     0.000000    0.000000   
...                                           ...         ...   
2022Q3  688799.SH 医药                     0.333809    0.920835   
        688800.SH 电子                    -0.312493    1.076427   
        688819.SH 电力设备及新能源               0.031121    1.418477   
        688981.SH 电子                     0.246840    0.554309   
        689009.SH 机械                    -0.666340    0.678782   

                            F_SUPPORT_SUBSIDY_mean  ...  \
QUARTER SECU_CODE INDUSTRY                          ...   
2011Q1  000009.SZ 综合                 -4.016420e-18  ...   
        000012.SZ 建材                 -4.016420e-18  ...   
        000016.SZ 家电                 -4.016420e-18  ...   
        000019.SZ 农林牧渔               -4.016420e-18  ...   
        000020.SZ 电子                 -4.016420e-18  ...   
...                                            ..

In [11]:
df_gov = pd.merge(
    left=df_gov, right=df[["F_GOV"]], left_index=True, right_index=True, how="outer"
)
df_gov


F_SUPPORT_SUBSIDY  F_DEPENDENCE_SUBSIDY  \
QUARTER SECU_CODE INDUSTRY                                            
2011Q1  000009.SZ 综合                 1.458430             -0.543251   
        000012.SZ 建材                 1.629855             -0.437518   
        000016.SZ 家电                 1.497990              2.564561   
        000019.SZ 农林牧渔              -0.461529              2.564561   
        000020.SZ 电子                -1.517770             -0.547717   
...                                       ...                   ...   
2022Q3  688799.SH 医药                -0.114306             -0.570632   
        688800.SH 电子                -1.103734             -0.719448   
        688819.SH 电力设备及新能源           1.546257             -0.420753   
        688981.SH 电子                 1.696215             -0.365687   
        689009.SH 机械                -1.644024             -0.773081   

                            F_SUBSIDY  F_SUPPORT_OTHER  F_DEPENDENCE_OTHER  \
QUARTER SECU_CODE INDUSTRY                                                   
2011Q1  000009.SZ 综合         2.001681              NaN                 NaN   
        000012.SZ 建材         2.067373              NaN                 NaN   
        000016.SZ 家电        -1.066571              NaN                 NaN   
        000019.SZ 农林牧渔      -3.026089              NaN                 NaN   
        000020.SZ 电子        -0.970052              NaN                 NaN   
...                               ...              ...                 ...   
2022Q3  688799.SH 医药         0.456326        -0.146204           -0.615050   
        688800.SH 电子        -0.384286        -1.148131           -0.767875   
        688819.SH 电力设备及新能源   1.967011         1.506474           -0.471376   
        688981.SH 电子         2.061902         1.675612           -0.416715   
        689009.SH 机械        -0.870943        -0.861457           -0.796988   

                             F_OTHER  F_SUPPORT_DEFERRED  \
QUARTER SECU_CODE INDUSTRY                                 
2011Q1  000009.SZ 综合             NaN                 NaN   
        000012.SZ 建材             NaN                 NaN   
        000016.SZ 家电             NaN                 NaN   
        000019.SZ 农林牧渔           NaN                 NaN   
        000020.SZ 电子             NaN                 NaN   
...                              ...                 ...   
2022Q3  688799.SH 医药        0.468846            1.198285   
        688800.SH 电子       -0.380256            0.698050   
        688819.SH 电力设备及新能源  1.977850            1.362780   
        688981.SH 电子        2.092327            0.767223   
        689009.SH 机械       -0.064468           -0.029103   

                            F_DEPENDENCE_DEFERRED  F_DEFERRED     F_GOV  
QUARTER SECU_CODE INDUSTRY                                               
2011Q1  000009.SZ 综合                          NaN         NaN  1.551145  
        000012.SZ 建材                          NaN         NaN  1.602051  
        000016.SZ 家电                          NaN         NaN -0.826508  
        000019.SZ 农林牧渔                        NaN         NaN -2.344981  
        000020.SZ 电子                          NaN         NaN -0.751714  
...                                           ...         ...       ...  
2022Q3  688799.SH 医药                     0.333809    0.864475  1.586585  
        688800.SH 电子                    -0.312493    1.010544  0.526282  
        688819.SH 电力设备及新能源               0.031121    1.331659  4.257132  
        688981.SH 电子                     0.246840    0.520383  3.543645  
        689009.SH 机械                    -0.666340    0.637237  0.006605  

[113812 rows x 10 columns]

In [12]:
df = df_gov.reset_index()
df


,QUARTER,SECU_CODE,INDUSTRY,F_SUPPORT_SUBSIDY,F_DEPENDENCE_SUBSIDY,F_SUBSIDY,F_SUPPORT_OTHER,F_DEPENDENCE_OTHER,F_OTHER,F_SUPPORT_DEFERRED,F_DEPENDENCE_DEFERRED,F_DEFERRED,F_GOV
0,2011Q1,000009.SZ,综合,1.458430,-0.543251,2.001681,NaN,NaN,NaN,NaN,NaN,NaN,1.551145
1,2011Q1,000012.SZ,建材,1.629855,-0.437518,2.067373,NaN,NaN,NaN,NaN,NaN,NaN,1.602051
2,2011Q1,000016.SZ,家电,1.497990,2.564561,-1.066571,NaN,NaN,NaN,NaN,NaN,NaN,-0.826508
3,2011Q1,000019.SZ,农林牧渔,-0.461529,2.564561,-3.026089,NaN,NaN,NaN,NaN,NaN,NaN,-2.344981
4,2011Q1,000020.SZ,电子,-1.517770,-0.547717,-0.970052,NaN,NaN,NaN,NaN,NaN,NaN,-0.751714
...,...,...,...,...,...,...,...,...,...,...,...,...,...
113807,2022Q3,688799.SH,医药,-0.114306,-0.570632,0.456326,-0.146204,-0.615050,0.468846,1.198285,0.333809,0.864475,1.586585
113808,2022Q3,688800.SH,电子,-1.103734,-0.719448,-0.384286,-1.148131,-0.767875,-0.380256,0.698050,-0.312493,1.010544,0.526282
113809,2022Q3,688819.SH,电力设备及新能源,1.546257,-0.420753,1.967011,1.506474,-0.471376,1.977850,1.362780,0.031121,1.331659,4.257132
113810,2022Q3,688981.SH,电子,1.696215,-0.365687,2.061902,1.675612,-0.416715,2.092327,0.767223,0.246840,0.520383,3.543645


# Modify Format of Date

In [15]:
col = []
for item in df["QUARTER"]:
    year = item[:4]  # year
    if item[5:] == "2":
        month = (
            int(item[5:]) * 3 + 3
        )  # month -the first month after half year statement is released
    else:
        month = (
            int(item[5:]) * 3 + 2
        )  # month -the first month after quarter statement is released
    i = year + "-" + str(month)
    i = datetime.datetime.strptime(i, "%Y-%m")
    col.append(i)

df["QUARTER"] = col
df

,QUARTER,SECU_CODE,INDUSTRY,F_SUPPORT_SUBSIDY,F_DEPENDENCE_SUBSIDY,F_SUBSIDY,F_SUPPORT_OTHER,F_DEPENDENCE_OTHER,F_OTHER,F_SUPPORT_DEFERRED,F_DEPENDENCE_DEFERRED,F_DEFERRED,F_GOV
0,2011-05-01,000009.SZ,综合,1.458430,-0.543251,2.001681,NaN,NaN,NaN,NaN,NaN,NaN,1.551145
1,2011-05-01,000012.SZ,建材,1.629855,-0.437518,2.067373,NaN,NaN,NaN,NaN,NaN,NaN,1.602051
2,2011-05-01,000016.SZ,家电,1.497990,2.564561,-1.066571,NaN,NaN,NaN,NaN,NaN,NaN,-0.826508
3,2011-05-01,000019.SZ,农林牧渔,-0.461529,2.564561,-3.026089,NaN,NaN,NaN,NaN,NaN,NaN,-2.344981
4,2011-05-01,000020.SZ,电子,-1.517770,-0.547717,-0.970052,NaN,NaN,NaN,NaN,NaN,NaN,-0.751714
...,...,...,...,...,...,...,...,...,...,...,...,...,...
113807,2022-11-01,688799.SH,医药,-0.114306,-0.570632,0.456326,-0.146204,-0.615050,0.468846,1.198285,0.333809,0.864475,1.586585
113808,2022-11-01,688800.SH,电子,-1.103734,-0.719448,-0.384286,-1.148131,-0.767875,-0.380256,0.698050,-0.312493,1.010544,0.526282
113809,2022-11-01,688819.SH,电力设备及新能源,1.546257,-0.420753,1.967011,1.506474,-0.471376,1.977850,1.362780,0.031121,1.331659,4.257132
113810,2022-11-01,688981.SH,电子,1.696215,-0.365687,2.061902,1.675612,-0.416715,2.092327,0.767223,0.246840,0.520383,3.543645


In [16]:
col = []
map ={}
quarter_map = {}
for i in df_return["DATE"]:
    quarter = i[:7]
    quarter_date = datetime.datetime.strptime(quarter,"%Y-%m") # get the corresponding date in df_gov
    date = datetime.datetime.strptime(i,"%Y-%m-%d") # get the time format to be replace date in df_return
    col.append(date)
    if quarter_date not in quarter_map:
        quarter_map[quarter_date] = date # create mapping
df_return["DATE"] = col
df_return.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 727030 entries, 0 to 727029
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   DATE       727030 non-null  datetime64[ns]
 1   SECU_CODE  727030 non-null  object        
 2   RETURN     430692 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 16.6+ MB


In [17]:
df = df.rename(columns={"QUARTER":"DATE"})
df_gov = df.set_index("DATE").rename(index=quarter_map)
df_gov

,SECU_CODE,INDUSTRY,F_SUPPORT_SUBSIDY,F_DEPENDENCE_SUBSIDY,F_SUBSIDY,F_SUPPORT_OTHER,F_DEPENDENCE_OTHER,F_OTHER,F_SUPPORT_DEFERRED,F_DEPENDENCE_DEFERRED,F_DEFERRED,F_GOV
DATE,,,,,,,,,,,,
2011-05-03,000009.SZ,综合,1.458430,-0.543251,2.001681,NaN,NaN,NaN,NaN,NaN,NaN,1.551145
2011-05-03,000012.SZ,建材,1.629855,-0.437518,2.067373,NaN,NaN,NaN,NaN,NaN,NaN,1.602051
2011-05-03,000016.SZ,家电,1.497990,2.564561,-1.066571,NaN,NaN,NaN,NaN,NaN,NaN,-0.826508
2011-05-03,000019.SZ,农林牧渔,-0.461529,2.564561,-3.026089,NaN,NaN,NaN,NaN,NaN,NaN,-2.344981
2011-05-03,000020.SZ,电子,-1.517770,-0.547717,-0.970052,NaN,NaN,NaN,NaN,NaN,NaN,-0.751714
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-01,688799.SH,医药,-0.114306,-0.570632,0.456326,-0.146204,-0.615050,0.468846,1.198285,0.333809,0.864475,1.586585
2022-11-01,688800.SH,电子,-1.103734,-0.719448,-0.384286,-1.148131,-0.767875,-0.380256,0.698050,-0.312493,1.010544,0.526282
2022-11-01,688819.SH,电力设备及新能源,1.546257,-0.420753,1.967011,1.506474,-0.471376,1.977850,1.362780,0.031121,1.331659,4.257132


In [18]:
df=df_gov.reset_index()
df =df[df["SECU_CODE"]!=0]
df

,DATE,SECU_CODE,INDUSTRY,F_SUPPORT_SUBSIDY,F_DEPENDENCE_SUBSIDY,F_SUBSIDY,F_SUPPORT_OTHER,F_DEPENDENCE_OTHER,F_OTHER,F_SUPPORT_DEFERRED,F_DEPENDENCE_DEFERRED,F_DEFERRED,F_GOV
0,2011-05-03,000009.SZ,综合,1.458430,-0.543251,2.001681,NaN,NaN,NaN,NaN,NaN,NaN,1.551145
1,2011-05-03,000012.SZ,建材,1.629855,-0.437518,2.067373,NaN,NaN,NaN,NaN,NaN,NaN,1.602051
2,2011-05-03,000016.SZ,家电,1.497990,2.564561,-1.066571,NaN,NaN,NaN,NaN,NaN,NaN,-0.826508
3,2011-05-03,000019.SZ,农林牧渔,-0.461529,2.564561,-3.026089,NaN,NaN,NaN,NaN,NaN,NaN,-2.344981
4,2011-05-03,000020.SZ,电子,-1.517770,-0.547717,-0.970052,NaN,NaN,NaN,NaN,NaN,NaN,-0.751714
...,...,...,...,...,...,...,...,...,...,...,...,...,...
113807,2022-11-01,688799.SH,医药,-0.114306,-0.570632,0.456326,-0.146204,-0.615050,0.468846,1.198285,0.333809,0.864475,1.586585
113808,2022-11-01,688800.SH,电子,-1.103734,-0.719448,-0.384286,-1.148131,-0.767875,-0.380256,0.698050,-0.312493,1.010544,0.526282
113809,2022-11-01,688819.SH,电力设备及新能源,1.546257,-0.420753,1.967011,1.506474,-0.471376,1.977850,1.362780,0.031121,1.331659,4.257132
113810,2022-11-01,688981.SH,电子,1.696215,-0.365687,2.061902,1.675612,-0.416715,2.092327,0.767223,0.246840,0.520383,3.543645


In [19]:
df = df.dropna(subset=["SECU_CODE","DATE"]).set_index(["SECU_CODE", "DATE"])
df_gov = df.sort_index()
df_gov

INDUSTRY  F_SUPPORT_SUBSIDY  F_DEPENDENCE_SUBSIDY  \
SECU_CODE DATE                                                           
000001.SZ 2018-05-02       银行           1.679060             -0.640474   
          2018-09-03       银行          -1.728443             -0.573927   
          2018-11-01       银行                NaN                   NaN   
          2019-05-06       银行                NaN                   NaN   
          2019-09-02       银行                NaN                   NaN   
...                       ...                ...                   ...   
689009.SH 2021-09-01       机械          -0.203771             -0.579712   
          2021-11-01       机械          -0.142155             -0.640276   
          2022-05-05       机械          -0.395100             -0.693611   
          2022-09-01       机械           0.518461             -0.644581   
          2022-11-01       机械          -1.644024             -0.773081   

                      F_SUBSIDY  F_SUPPORT_OTHER  F_DEPENDENCE_OTHER  \
SECU_CODE DATE                                                         
000001.SZ 2018-05-02   2.319534         1.523859           -0.678091   
          2018-09-03  -1.154516         0.093006           -0.608280   
          2018-11-01        NaN         1.692286           -0.644777   
          2019-05-06        NaN         1.591986           -0.708332   
          2019-09-02        NaN         1.709753           -0.616028   
...                         ...              ...                 ...   
689009.SH 2021-09-01   0.375941        -0.165463           -0.614049   
          2021-11-01   0.498121        -0.018805           -0.655575   
          2022-05-05   0.298512        -0.398289           -0.711170   
          2022-09-01   1.163042        -0.223592           -0.736066   
          2022-11-01  -0.870943        -0.861457           -0.796988   

                       F_OTHER  F_SUPPORT_DEFERRED  F_DEPENDENCE_DEFERRED  \
SECU_CODE DATE                                                              
000001.SZ 2018-05-02  2.201950                 NaN                    NaN   
          2018-09-03  0.701286                 NaN                    NaN   
          2018-11-01  2.337064                 NaN                    NaN   
          2019-05-06  2.300318                 NaN                    NaN   
          2019-09-02  2.325782                 NaN                    NaN   
...                        ...                 ...                    ...   
689009.SH 2021-09-01  0.448586            0.072442              -0.806341   
          2021-11-01  0.636770            0.094944              -0.612113   
          2022-05-05  0.312880            0.070687              -0.117234   
          2022-09-01  0.512474           -0.012079              -0.809897   
          2022-11-01 -0.064468           -0.029103              -0.666340   

                      F_DEFERRED     F_GOV  
SECU_CODE DATE                              
000001.SZ 2018-05-02         NaN  3.402496  
          2018-09-03         NaN -0.336594  
          2018-11-01         NaN  1.761719  
          2019-05-06         NaN  1.734933  
          2019-09-02         NaN  1.769699  
...                          ...       ...  
689009.SH 2021-09-01    0.878784  1.682826  
          2021-11-01    0.707057  1.617022  
          2022-05-05    0.187921  0.641072  
          2022-09-01    0.797818  2.088808  
          2022-11-01    0.637237  0.006605  

[113682 rows x 11 columns]

# Plug in Return & Frequency Conversion

In [20]:
df_return = df_return[
    (df_return["DATE"] < datetime.datetime.strptime("2022-12", "%Y-%m"))
    & (df_return["DATE"] > datetime.datetime.strptime("2011-05", "%Y-%m"))
]  # select time period
df_return

,DATE,SECU_CODE,RETURN
20056,2011-05-03,000001.SZ,-0.038877
20057,2011-05-03,000002.SZ,-0.067087
20058,2011-05-03,000004.SZ,-0.060574
20059,2011-05-03,000005.SZ,NaN
20060,2011-05-03,000006.SZ,-0.102890
...,...,...,...
716997,2022-11-01,688799.SH,-0.042589
716998,2022-11-01,688800.SH,0.015990
716999,2022-11-01,688819.SH,-0.062287
717000,2022-11-01,688981.SH,-0.013966


In [21]:
df_return = df_return.set_index(["SECU_CODE", "DATE"]).sort_index()
df_return

RETURN
SECU_CODE DATE                
000001.SZ 2011-05-03 -0.038877
          2011-06-01 -0.026965
          2011-07-01  0.017606
          2011-08-01 -0.026655
          2011-09-01 -0.047261
...                        ...
689009.SH 2022-07-01  0.104541
          2022-08-01 -0.165923
          2022-09-01 -0.113239
          2022-10-10 -0.163406
          2022-11-01  0.042174

[696946 rows x 1 columns]

## Frequency Conversion

In [22]:
df = pd.merge(left=df_gov, right=df_return, left_index=True, right_index=True, how = "right")
df

INDUSTRY  F_SUPPORT_SUBSIDY  F_DEPENDENCE_SUBSIDY  \
SECU_CODE DATE                                                           
000001.SZ 2011-05-03      NaN                NaN                   NaN   
          2011-06-01      NaN                NaN                   NaN   
          2011-07-01      NaN                NaN                   NaN   
          2011-08-01      NaN                NaN                   NaN   
          2011-09-01      NaN                NaN                   NaN   
...                       ...                ...                   ...   
689009.SH 2022-07-01      NaN                NaN                   NaN   
          2022-08-01      NaN                NaN                   NaN   
          2022-09-01       机械           0.518461             -0.644581   
          2022-10-10      NaN                NaN                   NaN   
          2022-11-01       机械          -1.644024             -0.773081   

                      F_SUBSIDY  F_SUPPORT_OTHER  F_DEPENDENCE_OTHER  \
SECU_CODE DATE                                                         
000001.SZ 2011-05-03        NaN              NaN                 NaN   
          2011-06-01        NaN              NaN                 NaN   
          2011-07-01        NaN              NaN                 NaN   
          2011-08-01        NaN              NaN                 NaN   
          2011-09-01        NaN              NaN                 NaN   
...                         ...              ...                 ...   
689009.SH 2022-07-01        NaN              NaN                 NaN   
          2022-08-01        NaN              NaN                 NaN   
          2022-09-01   1.163042        -0.223592           -0.736066   
          2022-10-10        NaN              NaN                 NaN   
          2022-11-01  -0.870943        -0.861457           -0.796988   

                       F_OTHER  F_SUPPORT_DEFERRED  F_DEPENDENCE_DEFERRED  \
SECU_CODE DATE                                                              
000001.SZ 2011-05-03       NaN                 NaN                    NaN   
          2011-06-01       NaN                 NaN                    NaN   
          2011-07-01       NaN                 NaN                    NaN   
          2011-08-01       NaN                 NaN                    NaN   
          2011-09-01       NaN                 NaN                    NaN   
...                        ...                 ...                    ...   
689009.SH 2022-07-01       NaN                 NaN                    NaN   
          2022-08-01       NaN                 NaN                    NaN   
          2022-09-01  0.512474           -0.012079              -0.809897   
          2022-10-10       NaN                 NaN                    NaN   
          2022-11-01 -0.064468           -0.029103              -0.666340   

                      F_DEFERRED     F_GOV    RETURN  
SECU_CODE DATE                                        
000001.SZ 2011-05-03         NaN       NaN -0.038877  
          2011-06-01         NaN       NaN -0.026965  
          2011-07-01         NaN       NaN  0.017606  
          2011-08-01         NaN       NaN -0.026655  
          2011-09-01         NaN       NaN -0.047261  
...                          ...       ...       ...  
689009.SH 2022-07-01         NaN       NaN  0.104541  
          2022-08-01         NaN       NaN -0.165923  
          2022-09-01    0.797818  2.088808 -0.113239  
          2022-10-10         NaN       NaN -0.163406  
          2022-11-01    0.637237  0.006605  0.042174  

[696946 rows x 12 columns]

In [23]:
df_last_month = df[
    (df.index.get_level_values(1).month == 11)
    & (df.index.get_level_values(1).year == 2022)
]
df_last_month

,,INDUSTRY,F_SUPPORT_SUBSIDY,F_DEPENDENCE_SUBSIDY,F_SUBSIDY,F_SUPPORT_OTHER,F_DEPENDENCE_OTHER,F_OTHER,F_SUPPORT_DEFERRED,F_DEPENDENCE_DEFERRED,F_DEFERRED,F_GOV,RETURN
SECU_CODE,DATE,,,,,,,,,,,,
000001.SZ,2022-11-01,银行,NaN,NaN,NaN,1.020561,-0.820711,1.841272,NaN,NaN,NaN,1.327059,0.240062
000002.SZ,2022-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.399767
000004.SZ,2022-11-01,计算机,-1.680043,1.717157,-3.397201,-1.556642,1.650817,-3.207459,-1.494376,1.266174,-2.760550,-7.692888,0.131874
000005.SZ,2022-11-01,电力及公用事业,-1.052278,-0.430442,-0.621837,-1.333036,-0.480993,-0.852043,NaN,NaN,NaN,-1.060839,0.130289
000006.SZ,2022-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.551685
...,...,...,...,...,...,...,...,...,...,...,...,...,...
688799.SH,2022-11-01,医药,-0.114306,-0.570632,0.456326,-0.146204,-0.615050,0.468846,1.198285,0.333809,0.864475,1.586585,-0.042589
688800.SH,2022-11-01,电子,-1.103734,-0.719448,-0.384286,-1.148131,-0.767875,-0.380256,0.698050,-0.312493,1.010544,0.526282,0.015990
688819.SH,2022-11-01,电力设备及新能源,1.546257,-0.420753,1.967011,1.506474,-0.471376,1.977850,1.362780,0.031121,1.331659,4.257132,-0.062287


In [24]:
df = df.drop(df_last_month.index) # delete last month

In [26]:
# extract Sep, Oct data
df_sep_oct = df[(df.index.get_level_values(1).month == 9)
    | (df.index.get_level_values(1).month == 10)]
df_sep_oct = df_sep_oct.fillna(method = "ffill", limit = 1) # fill Oct with Sep
df_sep_oct

INDUSTRY  F_SUPPORT_SUBSIDY  F_DEPENDENCE_SUBSIDY  \
SECU_CODE DATE                                                           
000001.SZ 2011-09-01      NaN                NaN                   NaN   
          2011-10-10      NaN                NaN                   NaN   
          2012-09-03      NaN                NaN                   NaN   
          2012-10-08      NaN                NaN                   NaN   
          2013-09-02      NaN                NaN                   NaN   
...                       ...                ...                   ...   
689009.SH 2020-10-09       机械          -0.948346             -0.574377   
          2021-09-01       机械          -0.203771             -0.579712   
          2021-10-08       机械          -0.203771             -0.579712   
          2022-09-01       机械           0.518461             -0.644581   
          2022-10-10       机械           0.518461             -0.644581   

                      F_SUBSIDY  F_SUPPORT_OTHER  F_DEPENDENCE_OTHER  \
SECU_CODE DATE                                                         
000001.SZ 2011-09-01        NaN              NaN                 NaN   
          2011-10-10        NaN              NaN                 NaN   
          2012-09-03        NaN              NaN                 NaN   
          2012-10-08        NaN              NaN                 NaN   
          2013-09-02        NaN              NaN                 NaN   
...                         ...              ...                 ...   
689009.SH 2020-10-09  -0.373969        -1.024039           -0.608155   
          2021-09-01   0.375941        -0.165463           -0.614049   
          2021-10-08   0.375941        -0.165463           -0.614049   
          2022-09-01   1.163042        -0.223592           -0.736066   
          2022-10-10   1.163042        -0.223592           -0.736066   

                       F_OTHER  F_SUPPORT_DEFERRED  F_DEPENDENCE_DEFERRED  \
SECU_CODE DATE                                                              
000001.SZ 2011-09-01       NaN                 NaN                    NaN   
          2011-10-10       NaN                 NaN                    NaN   
          2012-09-03       NaN                 NaN                    NaN   
          2012-10-08       NaN                 NaN                    NaN   
          2013-09-02       NaN                 NaN                    NaN   
...                        ...                 ...                    ...   
689009.SH 2020-10-09 -0.415883            0.667790               0.530198   
          2021-09-01  0.448586            0.072442              -0.806341   
          2021-10-08  0.448586            0.072442              -0.806341   
          2022-09-01  0.512474           -0.012079              -0.809897   
          2022-10-10  0.512474           -0.012079              -0.809897   

                      F_DEFERRED     F_GOV    RETURN  
SECU_CODE DATE                                        
000001.SZ 2011-09-01         NaN       NaN -0.047261  
          2011-10-10         NaN       NaN  0.079991  
          2012-09-03         NaN       NaN -0.083270  
          2012-10-08         NaN       NaN  0.013543  
          2013-09-02         NaN       NaN  0.110335  
...                          ...       ...       ...  
689009.SH 2020-10-09    0.137591 -0.447177       NaN  
          2021-09-01    0.878784  1.682826  0.075138  
          2021-10-08    0.878784  1.682826 -0.198702  
          2022-09-01    0.797818  2.088808 -0.113239  
          2022-10-10    0.797818  2.088808 -0.163406  

[120336 rows x 12 columns]

In [27]:
df_5_8 = df[
    (df.index.get_level_values(1).month == 5)
    | (df.index.get_level_values(1).month == 6)
    | (df.index.get_level_values(1).month == 7)
    | (df.index.get_level_values(1).month == 8)
]
df_5_8 = df_5_8.fillna(method = "ffill", limit= 3) # fill June, July, August with May
df_5_8

INDUSTRY  F_SUPPORT_SUBSIDY  F_DEPENDENCE_SUBSIDY  \
SECU_CODE DATE                                                           
000001.SZ 2011-05-03      NaN                NaN                   NaN   
          2011-06-01      NaN                NaN                   NaN   
          2011-07-01      NaN                NaN                   NaN   
          2011-08-01      NaN                NaN                   NaN   
          2012-05-02      NaN                NaN                   NaN   
...                       ...                ...                   ...   
689009.SH 2021-08-02       机械          -0.039202             -0.244508   
          2022-05-05       机械          -0.395100             -0.693611   
          2022-06-01       机械          -0.395100             -0.693611   
          2022-07-01       机械          -0.395100             -0.693611   
          2022-08-01       机械          -0.395100             -0.693611   

                      F_SUBSIDY  F_SUPPORT_OTHER  F_DEPENDENCE_OTHER  F_OTHER  \
SECU_CODE DATE                                                                  
000001.SZ 2011-05-03        NaN              NaN                 NaN      NaN   
          2011-06-01        NaN              NaN                 NaN      NaN   
          2011-07-01        NaN              NaN                 NaN      NaN   
          2011-08-01        NaN              NaN                 NaN      NaN   
          2012-05-02        NaN              NaN                 NaN      NaN   
...                         ...              ...                 ...      ...   
689009.SH 2021-08-02   0.205306         0.113172           -0.061047  0.17422   
          2022-05-05   0.298512        -0.398289           -0.711170  0.31288   
          2022-06-01   0.298512        -0.398289           -0.711170  0.31288   
          2022-07-01   0.298512        -0.398289           -0.711170  0.31288   
          2022-08-01   0.298512        -0.398289           -0.711170  0.31288   

                      F_SUPPORT_DEFERRED  F_DEPENDENCE_DEFERRED  F_DEFERRED  \
SECU_CODE DATE                                                                
000001.SZ 2011-05-03                 NaN                    NaN         NaN   
          2011-06-01                 NaN                    NaN         NaN   
          2011-07-01                 NaN                    NaN         NaN   
          2011-08-01                 NaN                    NaN         NaN   
          2012-05-02                 NaN                    NaN         NaN   
...                                  ...                    ...         ...   
689009.SH 2021-08-02            0.170986               0.810601   -0.639615   
          2022-05-05            0.070687              -0.117234    0.187921   
          2022-06-01            0.070687              -0.117234    0.187921   
          2022-07-01            0.070687              -0.117234    0.187921   
          2022-08-01            0.070687              -0.117234    0.187921   

                         F_GOV    RETURN  
SECU_CODE DATE                            
000001.SZ 2011-05-03       NaN -0.038877  
          2011-06-01       NaN -0.026965  
          2011-07-01       NaN  0.017606  
          2011-08-01       NaN -0.026655  
          2012-05-02       NaN -0.067879  
...                        ...       ...  
689009.SH 2021-08-02 -0.465154  0.122863  
          2022-05-05  0.641072  0.198764  
          2022-06-01  0.641072 -0.062285  
          2022-07-01  0.641072  0.104541  
          2022-08-01  0.641072 -0.165923  

[240672 rows x 12 columns]

In [28]:
df_11_4 = df[
    (df.index.get_level_values(1).month == 11)
    | (df.index.get_level_values(1).month == 12)
    | (df.index.get_level_values(1).month == 1)
    | (df.index.get_level_values(1).month == 2)
    | (df.index.get_level_values(1).month == 3)
    | (df.index.get_level_values(1).month == 4)
]
df_11_4 = df_11_4.fillna(method = "ffill", limit = 5) # fill all other month with Nov
df_11_4

INDUSTRY  F_SUPPORT_SUBSIDY  F_DEPENDENCE_SUBSIDY  \
SECU_CODE DATE                                                           
000001.SZ 2011-11-01      NaN                NaN                   NaN   
          2011-12-01      NaN                NaN                   NaN   
          2012-01-04      NaN                NaN                   NaN   
          2012-02-01      NaN                NaN                   NaN   
          2012-03-01      NaN                NaN                   NaN   
...                       ...                ...                   ...   
689009.SH 2021-12-01       机械          -0.142155             -0.640276   
          2022-01-04       机械          -0.142155             -0.640276   
          2022-02-07       机械          -0.142155             -0.640276   
          2022-03-01       机械          -0.142155             -0.640276   
          2022-04-01       机械          -0.142155             -0.640276   

                      F_SUBSIDY  F_SUPPORT_OTHER  F_DEPENDENCE_OTHER  F_OTHER  \
SECU_CODE DATE                                                                  
000001.SZ 2011-11-01        NaN              NaN                 NaN      NaN   
          2011-12-01        NaN              NaN                 NaN      NaN   
          2012-01-04        NaN              NaN                 NaN      NaN   
          2012-02-01        NaN              NaN                 NaN      NaN   
          2012-03-01        NaN              NaN                 NaN      NaN   
...                         ...              ...                 ...      ...   
689009.SH 2021-12-01   0.498121        -0.018805           -0.655575  0.63677   
          2022-01-04   0.498121        -0.018805           -0.655575  0.63677   
          2022-02-07   0.498121        -0.018805           -0.655575  0.63677   
          2022-03-01   0.498121        -0.018805           -0.655575  0.63677   
          2022-04-01   0.498121        -0.018805           -0.655575  0.63677   

                      F_SUPPORT_DEFERRED  F_DEPENDENCE_DEFERRED  F_DEFERRED  \
SECU_CODE DATE                                                                
000001.SZ 2011-11-01                 NaN                    NaN         NaN   
          2011-12-01                 NaN                    NaN         NaN   
          2012-01-04                 NaN                    NaN         NaN   
          2012-02-01                 NaN                    NaN         NaN   
          2012-03-01                 NaN                    NaN         NaN   
...                                  ...                    ...         ...   
689009.SH 2021-12-01            0.094944              -0.612113    0.707057   
          2022-01-04            0.094944              -0.612113    0.707057   
          2022-02-07            0.094944              -0.612113    0.707057   
          2022-03-01            0.094944              -0.612113    0.707057   
          2022-04-01            0.094944              -0.612113    0.707057   

                         F_GOV    RETURN  
SECU_CODE DATE                            
000001.SZ 2011-11-01       NaN -0.075456  
          2011-12-01       NaN -0.040012  
          2012-01-04       NaN  0.081416  
          2012-02-01       NaN  0.032742  
          2012-03-01       NaN -0.093656  
...                        ...       ...  
689009.SH 2021-12-01  1.617022  0.152806  
          2022-01-04  1.617022 -0.129967  
          2022-02-07  1.617022 -0.064009  
          2022-03-01  1.617022 -0.243667  
          2022-04-01  1.617022 -0.159478  

[330924 rows x 12 columns]

# Concatenate Data and Complete Frequency Conversion

In [29]:
df = pd.concat([df_sep_oct,df_5_8,df_11_4,df_last_month]).sort_index()
df

INDUSTRY  F_SUPPORT_SUBSIDY  F_DEPENDENCE_SUBSIDY  \
SECU_CODE DATE                                                           
000001.SZ 2011-05-03      NaN                NaN                   NaN   
          2011-06-01      NaN                NaN                   NaN   
          2011-07-01      NaN                NaN                   NaN   
          2011-08-01      NaN                NaN                   NaN   
          2011-09-01      NaN                NaN                   NaN   
...                       ...                ...                   ...   
689009.SH 2022-07-01       机械          -0.395100             -0.693611   
          2022-08-01       机械          -0.395100             -0.693611   
          2022-09-01       机械           0.518461             -0.644581   
          2022-10-10       机械           0.518461             -0.644581   
          2022-11-01       机械          -1.644024             -0.773081   

                      F_SUBSIDY  F_SUPPORT_OTHER  F_DEPENDENCE_OTHER  \
SECU_CODE DATE                                                         
000001.SZ 2011-05-03        NaN              NaN                 NaN   
          2011-06-01        NaN              NaN                 NaN   
          2011-07-01        NaN              NaN                 NaN   
          2011-08-01        NaN              NaN                 NaN   
          2011-09-01        NaN              NaN                 NaN   
...                         ...              ...                 ...   
689009.SH 2022-07-01   0.298512        -0.398289           -0.711170   
          2022-08-01   0.298512        -0.398289           -0.711170   
          2022-09-01   1.163042        -0.223592           -0.736066   
          2022-10-10   1.163042        -0.223592           -0.736066   
          2022-11-01  -0.870943        -0.861457           -0.796988   

                       F_OTHER  F_SUPPORT_DEFERRED  F_DEPENDENCE_DEFERRED  \
SECU_CODE DATE                                                              
000001.SZ 2011-05-03       NaN                 NaN                    NaN   
          2011-06-01       NaN                 NaN                    NaN   
          2011-07-01       NaN                 NaN                    NaN   
          2011-08-01       NaN                 NaN                    NaN   
          2011-09-01       NaN                 NaN                    NaN   
...                        ...                 ...                    ...   
689009.SH 2022-07-01  0.312880            0.070687              -0.117234   
          2022-08-01  0.312880            0.070687              -0.117234   
          2022-09-01  0.512474           -0.012079              -0.809897   
          2022-10-10  0.512474           -0.012079              -0.809897   
          2022-11-01 -0.064468           -0.029103              -0.666340   

                      F_DEFERRED     F_GOV    RETURN  
SECU_CODE DATE                                        
000001.SZ 2011-05-03         NaN       NaN -0.038877  
          2011-06-01         NaN       NaN -0.026965  
          2011-07-01         NaN       NaN  0.017606  
          2011-08-01         NaN       NaN -0.026655  
          2011-09-01         NaN       NaN -0.047261  
...                          ...       ...       ...  
689009.SH 2022-07-01    0.187921  0.641072  0.104541  
          2022-08-01    0.187921  0.641072 -0.165923  
          2022-09-01    0.797818  2.088808 -0.113239  
          2022-10-10    0.797818  2.088808 -0.163406  
          2022-11-01    0.637237  0.006605  0.042174  

[696946 rows x 12 columns]

In [30]:
df_gov = df.dropna(subset=["INDUSTRY"], axis = 0)
df_gov

INDUSTRY  F_SUPPORT_SUBSIDY  F_DEPENDENCE_SUBSIDY  \
SECU_CODE DATE                                                           
000001.SZ 2018-05-02       银行           1.679060             -0.640474   
          2018-06-01       银行           1.679060             -0.640474   
          2018-07-02       银行           1.679060             -0.640474   
          2018-08-01       银行           1.679060             -0.640474   
          2018-09-03       银行          -1.728443             -0.573927   
...                       ...                ...                   ...   
689009.SH 2022-07-01       机械          -0.395100             -0.693611   
          2022-08-01       机械          -0.395100             -0.693611   
          2022-09-01       机械           0.518461             -0.644581   
          2022-10-10       机械           0.518461             -0.644581   
          2022-11-01       机械          -1.644024             -0.773081   

                      F_SUBSIDY  F_SUPPORT_OTHER  F_DEPENDENCE_OTHER  \
SECU_CODE DATE                                                         
000001.SZ 2018-05-02   2.319534         1.523859           -0.678091   
          2018-06-01   2.319534         1.523859           -0.678091   
          2018-07-02   2.319534         1.523859           -0.678091   
          2018-08-01   2.319534         1.523859           -0.678091   
          2018-09-03  -1.154516         0.093006           -0.608280   
...                         ...              ...                 ...   
689009.SH 2022-07-01   0.298512        -0.398289           -0.711170   
          2022-08-01   0.298512        -0.398289           -0.711170   
          2022-09-01   1.163042        -0.223592           -0.736066   
          2022-10-10   1.163042        -0.223592           -0.736066   
          2022-11-01  -0.870943        -0.861457           -0.796988   

                       F_OTHER  F_SUPPORT_DEFERRED  F_DEPENDENCE_DEFERRED  \
SECU_CODE DATE                                                              
000001.SZ 2018-05-02  2.201950                 NaN                    NaN   
          2018-06-01  2.201950                 NaN                    NaN   
          2018-07-02  2.201950                 NaN                    NaN   
          2018-08-01  2.201950                 NaN                    NaN   
          2018-09-03  0.701286                 NaN                    NaN   
...                        ...                 ...                    ...   
689009.SH 2022-07-01  0.312880            0.070687              -0.117234   
          2022-08-01  0.312880            0.070687              -0.117234   
          2022-09-01  0.512474           -0.012079              -0.809897   
          2022-10-10  0.512474           -0.012079              -0.809897   
          2022-11-01 -0.064468           -0.029103              -0.666340   

                      F_DEFERRED     F_GOV    RETURN  
SECU_CODE DATE                                        
000001.SZ 2018-05-02         NaN  3.402496 -0.070211  
          2018-06-01         NaN  3.402496 -0.111345  
          2018-07-02         NaN  3.402496  0.080718  
          2018-08-01         NaN  3.402496  0.094276  
          2018-09-03         NaN -0.336594  0.084053  
...                          ...       ...       ...  
689009.SH 2022-07-01    0.187921  0.641072  0.104541  
          2022-08-01    0.187921  0.641072 -0.165923  
          2022-09-01    0.797818  2.088808 -0.113239  
          2022-10-10    0.797818  2.088808 -0.163406  
          2022-11-01    0.637237  0.006605  0.042174  

[423285 rows x 12 columns]

# Plug in Mkt_cap

In [ ]:
s